# Algo Trading Research Lab (VectorBT)

In this notebook, we explore the "Magnificent Seven" tech stocks. We will compare a simple Buy & Hold approach against a Technical Analysis (Momentum) strategy to see if algorithmic complexity actually adds value.

## 1. Setup & Data Acquisition

In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Configuration
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA']
benchmark = 'SPY'
start_date = '2021-01-01'
end_date = '2026-01-01'

vbt.settings.array_wrapper['freq'] = 'days'
vbt.settings.returns['year_freq'] = '252 days'
vbt.settings.plotting['layout']['template'] = 'vbt_dark'

# Download Data
print(f"Downloading data for {len(symbols)} assets...")
data = vbt.YFData.download(symbols, start=start_date, end=end_date)
price = data.get('Close')

# Plot normalized price (Growth of $1)
(price / price.iloc[0]).vbt.plot(yaxis_type='log').show()

## 2. The Baseline: Buy & Hold

Before we try fancy algorithms, let's see what happens if we just bought these stocks and held them.

In [2]:
# Create a "Buy" signal at the very first index, and never sell
entries_bh = pd.DataFrame.vbt.signals.empty_like(price)
entries_bh.iloc[0] = True
exits_bh = pd.DataFrame.vbt.signals.empty_like(price)

# Run Portfolio
pf_bh = vbt.Portfolio.from_signals(
    price, 
    entries_bh, 
    exits_bh,
    init_cash=10000 / len(symbols), # Equal allocation to each asset
    fees=0.001 # 0.1% transaction fee
)

print(f"Buy & Hold Total Return:\n{pf_bh.total_return().to_string()}")
print(f"\nAverage Return: {pf_bh.total_return().mean():.2%}")

Buy & Hold Total Return:
symbol
AAPL      1.155900
MSFT      1.312963
GOOGL     2.650361
AMZN      0.447231
NVDA     13.247732
META      1.469089
TSLA      0.846900

Average Return: 301.86%


In [3]:
# Shows how the portfolio composition changes over time
pf_bh_asset_value = pf_bh.asset_value(group_by=False)

# Show asset development over time
fig = pf_bh_asset_value.vbt.plot(
    trace_names=symbols,
    trace_kwargs=dict(stackgroup='one')
)
fig.show()

## 2b. The "Real" Baseline: Monthly Savings Plan (DCA)

Instead of investing $10k at once, let's invest $1,000 every month.

In [4]:
# Create a mask for the first day of every month
month_mask = ~price.index.to_period('M').duplicated()

# Define size: our initial cash divided by the number of assets
# and divided by the number of months we will be investing
dca_size = np.full_like(price, np.nan)
dca_size[month_mask] = 10000 / len(symbols) / sum(month_mask)

pf_dca = vbt.Portfolio.from_orders(
    price, 
    dca_size, 
    size_type='value', 
    init_cash=10000 / len(symbols),
    fees=0.001,
)

print(f"DCA Total Return: {pf_dca.total_return().mean():.2%}")
# Plot the value growth over time
fig = pf_dca.asset_value().sum(axis=1).vbt.plot(trace_kwargs=dict(name='DCA (Assets)'))
pf_dca.cash().sum(axis=1).vbt.plot(trace_kwargs=dict(name='DCA (Cash)'), fig=fig)
pf_dca.value().sum(axis=1).vbt.plot(trace_kwargs=dict(name='DCA (Value)'), fig=fig)
fig.show()

/tmp/ipykernel_26724/1442079039.py:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



DCA Total Return: 145.06%


## 3. Testing the Moving Average Crossover Strategy

Now, let's try to beat the market using a classic Golden Cross strategy.

- Buy when Fast MA crosses above Slow MA.
- Sell when Fast MA crosses below Slow MA.

In [5]:
# 1. Define Parameters
fast_window = 10
slow_window = 50

# 2. Calculate Indicators
fast_ma = vbt.MA.run(price, fast_window, short_name='fast')
slow_ma = vbt.MA.run(price, slow_window, short_name='slow')

# 3. Generate Signals
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

# 4. Run Backtest
pf_ma = vbt.Portfolio.from_signals(
    price, 
    entries, 
    exits, 
    init_cash=10000,
    fees=0.001
)

# 5. Visualize Trades for NVDA (Example)
fig = price['NVDA'].vbt.plot(trace_kwargs=dict(name='Close'))
pf_ma[fast_window, slow_window, 'NVDA'].positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)
fig.show()

print(f"MA Strategy Average Return: {pf_ma.total_return().mean():.2%}")

MA Strategy Average Return: 128.86%


## 4. Hyperparameter Optimization

We might think: "Maybe 10/50 isn't the best combination. Let's test ALL combinations!"

VectorBT allows us to test thousands of strategies in seconds using Broadcasting.

In [6]:
# Define a range of windows to test
windows = np.arange(10, 50, step=2) # Test 10, 12, 14... up to 48

# Run Combinations (Cartesian Product)
# This runs every window against every other window
fast_ma, slow_ma = vbt.MA.run_combs(price, windows, r=2, short_names=['fast', 'slow'])

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_opt = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.001, freq='1D')

print(f"Tested {len(windows) * (len(windows)-1)} parameter combinations across {len(symbols)} assets.")

Tested 380 parameter combinations across 7 assets.


## 5. Visualizing the Optimization (Heatmap)

This heatmap shows the Sharpe Ratio for every combination of Fast/Slow windows.

- Yellow/Red: High performance.
- Blue: Low performance.

_Note: If you see scattered yellow spots, it's likely noise (overfitting). If you see large yellow regions, the strategy is robust._

In [7]:
# Aggregate Sharpe Ratio across all assets
mean_sharpe = pf_opt.sharpe_ratio().groupby(['fast_window', 'slow_window']).mean()

# Plot Heatmap
fig = mean_sharpe.vbt.heatmap(
    x_level='fast_window', 
    y_level='slow_window',
    symmetric=True
)
fig.show()

# Find the absolute best parameters
best_params = mean_sharpe.idxmax()
print(f"Best Parameters found: Fast={best_params[0]}, Slow={best_params[1]}")
print(f"Sharpe at best params: {mean_sharpe.max():.2f}")

/opt/conda/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



Best Parameters found: Fast=14, Slow=36
Sharpe at best params: 0.70


## 6. Strategy vs. Benchmark

Let's compare the Best Found Strategy against the simple Buy & Hold.

In [8]:
# Get the portfolio for the best parameters
pf_best = pf_opt.xs(best_params, level=['fast_window', 'slow_window'])

# Compare Total Returns
comparison = pd.DataFrame({
    'Buy & Hold': pf_bh.total_return(),
    'Best MA Strategy': pf_best.total_return()
})

print(comparison)

# Plot Equity Curves (Average of all assets)
fig = pf_bh.value().sum(axis=1).vbt.plot(trace_kwargs=dict(name='Buy & Hold (Portfolio)'))
pf_best.value().sum(axis=1).vbt.plot(trace_kwargs=dict(name='Best MA Strategy (Portfolio)'), fig=fig)
fig.show()

        Buy & Hold  Best MA Strategy
symbol                              
AAPL      1.155900          0.341885
MSFT      1.312963          0.553787
GOOGL     2.650361          0.980479
AMZN      0.447231          0.660744
NVDA     13.247732          3.226047
META      1.469089          2.053393
TSLA      0.846900          1.791309


## 6. Periodic Rebalancing

Instead of timing the market, let's just keep the 7 giants equally weighted.

Rebalance every quarter.

In [9]:
# Monthly rebalancing
size = np.full_like(price, np.nan)
mask = ~price.index.to_period('Q').duplicated()
size[mask, :] = [1 / len(symbols)] * len(symbols)
pf_rebal = vbt.Portfolio.from_orders(price, size, size_type='targetpercent', cash_sharing=True, init_cash=10000, fees=0.001)

/tmp/ipykernel_26724/3336484718.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [10]:
# Shows how the portfolio composition changes over time
rb_asset_value = pf_rebal.asset_value(group_by=False)

# Show asset development over time
fig = rb_asset_value.vbt.plot(
    trace_names=symbols,
    trace_kwargs=dict(stackgroup='one')
)
fig.show()

In [11]:
# Normalize to percentage (0 to 1), with cash being the remaining part
fig = (rb_asset_value.vbt / pf_rebal.value()).vbt.plot(
    trace_names=symbols,
    trace_kwargs=dict(stackgroup='one')
)
fig.show()

In [12]:
# Compare Total Returns
comparison = pd.DataFrame({
    'Buy & Hold': pf_bh.total_return(),
    'Rebalanced': pf_rebal.total_return()
})

print(comparison)

# Plot Equity Curves (Average of all assets)
fig = pf_bh.value().sum(axis=1).vbt.plot(trace_kwargs=dict(name='Buy & Hold (Portfolio)'))
pf_rebal.value().vbt.plot(trace_kwargs=dict(name='Rebalanced Portfolio'), fig=fig)
fig.show()

        Buy & Hold  Rebalanced
symbol                        
AAPL      1.155900    2.436469
MSFT      1.312963    2.436469
GOOGL     2.650361    2.436469
AMZN      0.447231    2.436469
NVDA     13.247732    2.436469
META      1.469089    2.436469
TSLA      0.846900    2.436469
